In [306]:
#!pip install numpy, pandas
import numpy as np 
import pandas as pd
from datetime import datetime as dt, timedelta
from tabulate import tabulate
import re

In [307]:
path = "D:\\Downloads\\"
filename = 'restaraunts-opening-hour.xlsx'

## Read the data into the notebook

In [308]:
df = pd.read_excel(path+filename, header=None).rename(columns = {0:'restaurants', 1: 'open_timings'})

## Functions to display the structure and descriptive statistics of the dataset

In [309]:
def detect_outliers(df, c):
    p1 = df[c].quantile(0.25)
    p2 = df[c].quantile(0.75)
    iq = iqr(df[c])
    mask = ((df[c]<(p1-1.5*iq)) & (df[c]>(p2+1.5*iq)))
    if mask.sum()<1:
        mask = ((df[c]>df[c].quantile(0.01)) & (df[c]<df[c].quantile(0.999)))
        dft = df[mask]
    else: 
        dft = df[~mask]
        print(f'% of Outliers in {c}: ', 100.0*(df.shape[0]-dft.shape[0])/df.shape[0], end='\n\n')

    return dft

def summarize_df(df: pd.DataFrame, treat_outliers=False):

    rows = df.shape[0]
    colnos = df.shape[1]
    print('No. of rows: ', rows, ', No of columns: ', colnos)

    dup = df.duplicated().sum()
    if dup>0:
        print('No. of duplicate rows detected: ', dup, end = '\n\n')

        df.drop_duplicates(inplace=True)
        print('Removed all duplicates', end='\n\n')

    elif dup==0:
        print('No duplicates in the dataset', end='\n\n')


    print('Columns with nan value percentages: ', end='\n\n')

    nan = df.isna().sum()
    print(100*nan[nan>0]/rows, end='\n\n')

    df.columns = [c.lower().replace(' ', '_') for c in df.columns]

    print('Datatypes of columns:', end='\n\n')

    tb = {'Datatype': [], 'unique_values': []}
    cols = []
    for c in df.columns:

        try:
            if 'datetime' in c or 'time' in c or 'timestamp' in c:
                df[c] = pd.to_datetime(df[c])

            elif df[c].dtype=='float64' or df[c].dtype=='int64' :

                if df[c].nunique()>200:
                    if treat_outliers==True:
                        df = detect_outliers(df, c)
                    else: 
                        _ = detect_outliers(df,c)

            tb['Datatype'].append(df[c].dtype)
            tb['unique_values'].append(df[c].nunique())
            cols.append(c)
        except Exception as e:
            print('Error:', e)

    tb = pd.DataFrame(tb, index = cols)

    print(tabulate(tb, headers = 'keys', tablefmt = 'psql'), end = '\n\n')

    print('Descriptive Stats: ', end='\n\n')
    print(tabulate(df.describe(include='all'), headers = 'keys', tablefmt = 'psql'))

    return df


In [310]:
_ = summarize_df(df)

No. of rows:  51 , No of columns:  2
No duplicates in the dataset

Columns with nan value percentages: 

Series([], dtype: float64)

Datatypes of columns:

+--------------+------------+-----------------+
|              | Datatype   |   unique_values |
|--------------+------------+-----------------|
| restaurants  | object     |              51 |
| open_timings | object     |              43 |
+--------------+------------+-----------------+

Descriptive Stats: 

+--------+---------------+-----------------------------------------------------------+
|        | restaurants   | open_timings                                              |
|--------+---------------+-----------------------------------------------------------|
| count  | 51            | 51                                                        |
| unique | 51            | 43                                                        |
| top    | Kushi Tsuru   | Mon-Thu, Sun 11:30 am - 10 pm  / Fri-Sat 11:30 am - 11 pm |
| freq   | 1

In [317]:
df1 = df.copy()

## Functions to build the schema with 2 tables, table1=restaurants table, table2= schedule

In [555]:
def convert_to_24H(x): #function replaces a given string of format 11 am to a form 11:00
    l = x.split(' ')
    x = l[0]
    
    #Convert the given string into datetime object
    try:
        x = dt.strptime(x, "%H:%M")
    except Exception: x = dt.strptime(x, "%H")
    
    if (l[-1] == 'am'):
        if x.hour==12:
            x += timedelta(hours = 12) #add 12 hours if the given time is 12 am to convert to 24:00
    elif (l[-1] == 'pm'):
        if x.hour!=12:
            x += timedelta(hours=12) #add 12 hours if the given time is 1 pm to 11 pm to convert into 24H format
    #else: print("Invalid Values, opening time can't be greater than closing time")
    
    return l, x, x.strftime("%H:%M")

def convert_to_24H_list(x): #function replaces a given string of format 11 am to 8 pm to a list of times like [11:00, 12:00,..]
    l = x.split(' - ')
    start, st, _ = convert_to_24H(l[0]) #starting of the range
    end, ed, _ = convert_to_24H(l[-1])#end of the range
    
    if (start[-1] == 'am') and (end[-1] == 'am'):
        if ed.hour==0:
            ed -= timedelta(minutes=1)
        elif ed.hour<st.hour:
            ed += timedelta(days=1)
    elif (start[-1] == 'pm') and (end[-1] == 'pm'):
        if st.hour!=12:
            st = st + timedelta(hours=12)
    elif (start[-1] == 'pm') and (end[-1] == 'am'):
        if ed.hour==0:
            ed -= timedelta(minutes=1)
        else:
            ed += timedelta(days=1)
    return [(st + timedelta(minutes=(x*30))).strftime("%H:%M") for x in range((((ed-st)/3600).seconds+1)*2) if (st + timedelta(minutes=(x*30)))<=ed]

def form_schedule(df: pd.DataFrame)->pd.DataFrame:
    
    #Assigning interpretable names to columns
    df.rename(columns = {0:'restaurants', 1: 'open_timings'}, inplace=True)
    
    #restructuring the dataframe in lexicographical order of restaurants and adding restaurant ids
    df.sort_values(by = 'restaurants', ignore_index=True, inplace=True)
    df['rest_id'] = df.index+1
    
    #Forming restaurants dimension table for ease of access
    restaurants = df[['rest_id', 'restaurants']].rename(columns = {'restaurants': 'rest_name'})
    
    #Extract weekdays and 
    df['weekdays'] = df['open_timings'].apply(lambda x: re.findall('(\w{3}\-*\w*)', x))
    df['timings'] = df['open_timings'].apply(lambda x: re.findall('(\d+:*\d* \w+ -* \d+:*\d* \w+)', x))
    
    weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    def replace_range(x): #function replaces a given string of format Mon-Thu to a list of weekdays [Mon, Tue, Wed, Thu]
        if '-' in x:
            l = x.split('-')
            start = weekdays.index(l[0])
            end = weekdays.index(l[-1])
            temp = []
            temp = [weekdays[i] for i in range(start,end+1,1)]
            return temp
    
    def collect_days(x): #function collects the days in a list of lists based on ranges or individual days 
        days = []
        for i, day in enumerate(x):
            if '-' in day:
                try: 
                    if len(days[i-1])==1:
                        days[i-1]+=replace_range(day)
                    else: days.append(replace_range(day))
                except Exception:
                    days.append(replace_range(day))
            elif (i>0 and i<(len(x)-1) and ('-' in x[i-1]) and ('-' in x[i+1])): 
                days[i-1] += [day]
            else: 
                days.append([day])
        return days
    
    df.weekdays = df.weekdays.apply(lambda x: collect_days(x))
    df.drop(columns = 'open_timings', inplace=True)
    
    df = df.explode(column = ['weekdays', 'timings']) #initial explode for converting list of lists to lists
    df.timings = df.timings.apply(lambda x: convert_to_24H_list(x)) #extract time ranges from strings
    t = df.explode(column = 'weekdays').explode(column='timings') #explode list of days and times for individual values
    t = t.groupby(['timings', 'weekdays'], as_index=False).agg({'rest_id': set}) #groupby times and weeks to collect list of restaurants based on group
    schedule = t.pivot(columns='weekdays', index='timings', values='rest_id') #form schedule data set showing open restaurants based on time and day
    
    return restaurants, schedule

In [414]:
#extract the restaurants table and schedules from the given data into 2 dataframes
restaurants, schedule = form_schedule(df.copy()) 

## Functions to get the list of restaurants open on the given weekday and time

In [590]:
def output_list_of_restaurants(weekday, time, rest=restaurants.copy(), schedule=schedule.copy()):
    
    _, _, time = convert_to_24H(time) #convert the input time to 24H format
    
    weekdays = {'Monday': 'Mon', 'Tuesday':'Tue', 'Wednesday':'Wed', 'Thursday':'Thu', 'Friday':'Fri', 'Saturday':'Sat', 'Sunday':'Sun'}
    weekday = weekday.title()
    if weekday not in schedule.columns:
        if weekday in weekdays.keys():
            weekday = weekdays[weekday] #replace given weekday string format, with the values inside the schedule table
    
    def get_listed_time(tm):
        for i, tim in enumerate(schedule.index):
            #time = dt.strptime(tim, "%H:%M")
            if tim>tm:
                return schedule.index[i-1] #if the time is lesser than a certain time, the time before that is chosen to pick the restaurants
                break
            elif tim==tm:
                return tim
                break
    
    rest1 = restaurants.set_index('rest_id')['rest_name'].to_dict()
    
    if time not in schedule.index:
        time = get_listed_time(time) 
    
    try:
        rest_ = list(schedule.loc[time, weekday])
        result = pd.Series(rest_).map(rest1) #replace restaurant ids with restaurant names
        headers = ['Restaurant-ID', 'Restaurant-Name']
        #print open restaurants with their ids in a table format
        #print(tabulate(list(zip(rest_, result.values)), headers = headers, tablefmt = 'psql'))
        return list(rest_)
    except Exception:
        return 'No Restaurants Open on the given day and time'
        
def take_input(day_time):
    print('Please Input the weekday and time in the same order, separated by a comma:')
    #day_time = input().strip()
    print('Open Restaurants are: ')
    l = day_time.split(',')
    weekday = l[0].strip()
    time = l[1].strip()
    return output_list_of_restaurants(weekday, time)

## Taking input here

In [578]:
#input = tuesday, 10 am
take_input()

Please Input the weekday and time in the same order, separated by a comma:
tuesday, 10 am
Open Restaurants are: 
+-----------------+-----------------------------------+
|   Restaurant-ID | Restaurant-Name                   |
|-----------------+-----------------------------------|
|               5 | All Season Restaurant             |
|              38 | Sabella & La Torre                |
|              41 | Santorini's Mediterranean Cuisine |
|              49 | Tong Palace                       |
|              18 | Herbivore                         |
|              19 | India Garden Restaurant           |
+-----------------+-----------------------------------+


[5, 38, 41, 49, 18, 19]

In [581]:
#input = monday, 5:30 pm
#converted input function to take arguments for ease of testing
take_input('monday, 5:30 pm')

Please Input the weekday and time in the same order, separated by a comma:
Open Restaurants are: 
+-----------------+-------------------------------------+
|   Restaurant-ID | Restaurant-Name                     |
|-----------------+-------------------------------------|
|               1 | 2G Japanese Brasserie               |
|               2 | A-1 Cafe Restaurant                 |
|               3 | Alhamra                             |
|               4 | Alioto's Restaurant                 |
|               5 | All Season Restaurant               |
|               6 | Bai Thong Thai Cuisine              |
|               7 | Bamboo Restaurant                   |
|               8 | Blu Restaurant                      |
|               9 | Bombay Indian Restaurant            |
|              10 | Bow Hon Restaurant                  |
|              11 | Burger Bar                          |
|              12 | Canton Seafood & Dim Sum Restaurant |
|              13 | Cesario's   

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [582]:
#input = friday, 7:30 am
take_input('friday, 7:30 am')

Please Input the weekday and time in the same order, separated by a comma:
Open Restaurants are: 
No Restaurants Open on the given day and time


In [583]:
#input = sunday, 1:30 am
take_input('sunday, 1:30 am')

Please Input the weekday and time in the same order, separated by a comma:
Open Restaurants are: 
+-----------------+-------------------------------+
|   Restaurant-ID | Restaurant-Name               |
|-----------------+-------------------------------|
|              25 | Marrakech Moroccan Restaurant |
|              28 | Naan 'N' Curry                |
+-----------------+-------------------------------+


[25, 28]

## Testing

In [576]:
#!pip install ipytest

In [586]:
import pytest
import ipytest
ipytest.autoconfig()

In [593]:
@pytest.mark.parametrize("day_time", ['tuesday, 10 am'])
def test_input1(day_time):
    result = take_input(day_time)
    assert sorted(result) == sorted([5, 38, 41, 49, 18, 19]), "Test Failed"

@pytest.mark.parametrize("day_time", ['monday, 5:30 pm'])
def test_input2(day_time):
    result = take_input(day_time)
    assert sorted(result) == sorted([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]), "Test Failed"

@pytest.mark.parametrize("day_time", ['friday, 7:30 am'])
def test_input3(day_time):
    result = take_input(day_time)
    assert result == 'No Restaurants Open on the given day and time', "Test Failed, there is no open restaurant in the data at the given time"
    
@pytest.mark.parametrize("day_time", ['sunday, 1:30 am'])
def test_input4(day_time):
    result = take_input(day_time)
    assert sorted(result) == sorted([25, 28]), "Test Failed"

In [594]:
ipytest.run()

....                                                                                         [100%]
4 passed in 0.05s


<ExitCode.OK: 0>